In [52]:
import pymysql
import pandas as pd

# 공통
%matplotlib inline
import numpy as np
import statsmodels.api as sm

# 예측 1
import matplotlib.pyplot as plt
import datetime
from dateutil.relativedelta import relativedelta
import seaborn as sns
from statsmodels.tsa.stattools import acf
from statsmodels.tsa.stattools import pacf
from statsmodels.tsa.seasonal import seasonal_decompose

# 예측 2
from scipy.stats import norm
from tqdm import tqdm
from sklearn.metrics import mean_squared_error

In [53]:
con = pymysql.connect(host = '127.0.0.1', user = 'root', password = 'kyjin0413!',
                       db = 'lpointsample', charset = 'utf8')
cur = con.cursor(pymysql.cursors.DictCursor)
cur.execute(
    "select de_dt as 날짜 , sum(buy_am) as 금액 from 엘페이이용_view group by de_dt order by de_dt")
result=cur.fetchall()
con.commit()
con.close()
df= pd.DataFrame(result)
# print(df.head(10))
df.head(10)

df.columns = ['date', 'price']
df = df.drop(['date'], axis = 1)
df.head(10)

,price
0,39209808
1,62239672
2,57365413
3,95785399
4,37415973
5,44468170
6,39459368
7,56869794
8,72284738
9,49765511


In [54]:
# 인덱스 지정(?)

# strptime은 날짜와 시간 형식의 문자열을 datetime으로 변환해 줌
# strftime은 날짜와 시간(datetime)을 문자열로 출력
start = datetime.datetime.strptime("2021-01-01", "%Y-%m-%d")

# 기준 날짜에 대한 연산이 필요할 때 dateutil.relativedelta모듈의 relativedelta함수 이용
df_list = [start + relativedelta(days = x) for x in range(0, len(df)) ]

df['index'] = df_list

# set_index는 데이터프레임을 먼저 불러온 후, 특정 열을 인덱스로 설정할 때 사용
df.set_index( ['index'], inplace=True) 

# index 출력 X
df.index.name=None
df.tail()

,price
2021-12-27,48002138
2021-12-28,38070210
2021-12-29,52093193
2021-12-30,47463341
2021-12-31,66341333


In [55]:
# price가 object임
df = df.astype('int')

In [56]:
df.dtypes

price    int32
dtype: object

In [ ]:
# coef값은 0에서 가장 떨어졌을때 가장 이상적
# p 값은 0과 가장 가까울때가 가장 좋습니다

mod = sm.tsa.statespace.SARIMAX(
    df['total'],
    order = (1, 1, 1),
    seasonal_order = (0, 1, 1, 12),
    # AR 매개 변수를 변환하여 모델의 자동 회귀 구성 요소에서 정상 성을 강제 적용할지 여부입니다. 기본값은 True입니다.
    # AR항이 stationary를 만족하게끔 강제하는 것
    enforce_stationarity = False,
    # 모델의 이동 평균 구성 요소에서 반전 성을 강제하기 위해 MA 매개 변수를 변환할지 여부입니다. 기본값은 True입니다.
    # MA항이 starionary를 만족하게끔 강제하는 것
    enforce_invertibility = False
)

results = mod.fit()

# print(results.summary())
print( results.summary().tables[1] )

In [ ]:
# Get forecast some steps ahead in future 미래 예측
pred_uc = results.get_forecast( steps = 6 )

In [18]:
forecast_mean = pred_uc.predicted_mean
forecast_mean

2021-12-01    6.983970e+07
2021-12-02    6.920729e+07
2021-12-03    7.021301e+07
2021-12-04    7.004778e+07
2021-12-05    7.015390e+07
2021-12-06    7.084773e+07
2021-12-07    7.088021e+07
2021-12-08    7.044509e+07
2021-12-09    6.956922e+07
2021-12-10    6.972534e+07
2021-12-11    7.036194e+07
2021-12-12    6.955451e+07
2021-12-13    6.955451e+07
2021-12-14    6.955451e+07
2021-12-15    6.955451e+07
2021-12-16    6.955451e+07
2021-12-17    6.955451e+07
2021-12-18    6.955451e+07
2021-12-19    6.955451e+07
2021-12-20    6.955451e+07
2021-12-21    6.955451e+07
2021-12-22    6.955451e+07
2021-12-23    6.955451e+07
2021-12-24    6.955451e+07
2021-12-25    6.955451e+07
2021-12-26    6.955451e+07
2021-12-27    6.955451e+07
2021-12-28    6.955451e+07
2021-12-29    6.955451e+07
2021-12-30    6.955451e+07
2021-12-31    6.955451e+07
Freq: D, Name: predicted_mean, dtype: float64

In [26]:
# forecast.to_csv('lpoint_days2.csv', header=['forecast'], index=False)